In [1]:
import tensorflow as tf
import os
from pathlib import Path
import urllib.request as request
import numpy as np
import matplotlib.pyplot as plt



In [4]:
IMAGE_SIZE = (224, 224)




In [5]:
model = tf.keras.applications.VGG16(
    include_top=True, weights="imagenet", input_tensor=None, input_shape=None, classes=1000 
)


553476096/553467096 [==============================] - 151s 0us/step


In [6]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
model.save("VGG16_full_model.h5")

In [9]:
model.layers[1].padding

'same'

In [14]:
data_URL = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"

# create a directory
def create_dirs(dir_path):
    os.makedirs(dir_path, exist_ok=True)
    print(f"{dir_path} directory created")
    
ROOT_DATA_DIR = "data"
create_dirs(ROOT_DATA_DIR)


data_zip_file = "data.zip"
data_zip_path = os.path.join(ROOT_DATA_DIR, data_zip_file)

if not os.path.isfile(data_zip_file):
    print("downloading data...")
    filename, headers = request.urlretrieve(data_URL, data_zip_path)
    print(f"filename: {filename} created with info \n{headers}")
else:
    print(f"file is already present")

data directory created
downloading data...
filename: data\data.zip created with info 
Content-Type: application/zip
Content-Length: 47286322
Connection: close
Last-Modified: Wed, 15 Mar 2017 18:46:00 GMT
x-amz-version-id: null
Accept-Ranges: bytes
Server: AmazonS3
Date: Sun, 27 Mar 2022 11:43:27 GMT
ETag: "5f8c32a6554f6acb4d649776e7735e48"
X-Cache: Hit from cloudfront
Via: 1.1 5c339b1b0b36f58990857e3126741c46.cloudfront.net (CloudFront)
X-Amz-Cf-Pop: BOM50-C1
X-Amz-Cf-Id: _0cTeScwZemlCDVbleYDEaEjz-pBE-_jQa38wawhMU3_L8ebtS1EuQ==
Age: 8376




In [15]:
# unzip data
from zipfile import ZipFile

unzip_data_dirname = "unzip_data_dir"
unzip_data_dir = os.path.join(ROOT_DATA_DIR,unzip_data_dirname)

if not os.path.exists(unzip_data_dir):
    os.makedirs(unzip_data_dir,exist_ok=True)
    with ZipFile(data_zip_path) as f:
        f.extractall(unzip_data_dir)
else:
    print(f"data already extracted")

In [16]:
main_data_dir = Path("./data/unzip_data_dir/hymenoptera_data/train")
main_data_dir

WindowsPath('data/unzip_data_dir/hymenoptera_data/train')

In [17]:
BATCH_SIZE=32
pixels=224
IMAGE_SIZE=(pixels,pixels)

In [18]:
datagen_kwargs= dict(rescale=1./255,validation_split=0.20)
dataflow_kwargs= dict(target_size=IMAGE_SIZE,batch_size=BATCH_SIZE)

In [20]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)


valid_generator = valid_datagen.flow_from_directory(
    main_data_dir, subset="validation", shuffle=False, **dataflow_kwargs)


do_data_augmentation = False

if do_data_augmentation:
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotational_range=40,
        horizontal_flip=True,
        width_shift_range=0.2, 
        height_shift_range=0.2, 
        shear_range=0.2, 
        zoom_range=0.2, 
        **datagen_kwargs)
else:
    train_datagen = valid_datagen

train_generator = train_datagen.flow_from_directory(
    main_data_dir, subset="training", shuffle=True, **dataflow_kwargs
)

Found 48 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [22]:
RGB_IMAGE_SIZE = (pixels, pixels, 3)


vgg = tf.keras.applications.vgg16.VGG16(
    input_shape=RGB_IMAGE_SIZE,
    weights="imagenet",
    include_top=False
)

58900480/58889256 [==============================] - 462s 8us/step


In [23]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
for layer in vgg.layers:
    print(f"{layer.name:20s}:{layer.trainable}")
    

input_2             :True
block1_conv1        :True
block1_conv2        :True
block1_pool         :True
block2_conv1        :True
block2_conv2        :True
block2_pool         :True
block3_conv1        :True
block3_conv2        :True
block3_conv3        :True
block3_pool         :True
block4_conv1        :True
block4_conv2        :True
block4_conv3        :True
block4_pool         :True
block5_conv1        :True
block5_conv2        :True
block5_conv3        :True
block5_pool         :True


In [25]:
vgg.output

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [26]:
del model

In [28]:
CLASSES = 2
x = tf.keras.layers.Flatten()(vgg.output)
prediction = tf.keras.layers.Dense(CLASSES, activation="softmax")(x)

model = tf.keras.models.Model(inputs=vgg.input, outputs = prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(
    optimizer=tf.keras.optimizer.SGD(learning_rate=0.01,momentum=0.9),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 10

histroy = model.fit(
    train_generator, epochs=EPOCHS, validation_data=valid_generator
)

In [ ]:
model.save("trained_model.h5")

In [ ]:
label_map={val:key for key, val in train_generator.class_indices.items()}
label_map

In [ ]:
test_img = plt.imread("./data/unzip_data_dir/hymenoptera_data/val/ants/800px-Meat_eater_ant_qeen_excavating_hole.jpg")

In [ ]:
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
model.predict(test_img)

In [ ]:
tf.image.resize(test_img,(224,224))

In [ ]:
model.predict(resized_img)

In [ ]:
input_data = tf.expand_dims(resized_img,axis=0)

In [ ]:
pred = model.predict(input_data)
pred

In [ ]:
argmax=tf.argmax(pred[0]).numpy()

In [ ]:
label_map[argmax]

In [ ]:
loaded_model = tf.keras.models.load_model("./trained_model.h5")

In [ ]:
preprocessd_img = tf.keras.applications.vgg16.preprocess_input(resized_img)

In [ ]:
preprocessd_img.shape

In [ ]:
plt.imshow(preprocessd_img)

In [ ]:
plt.imshow(test_img)

In [ ]:
full_model = 138_357_544
our_model 14_714_688

In [ ]:
type(full_model)

In [ ]:
our_model/full_model